In [2]:
# Import necessary libraries
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from category_encoders import OneHotEncoder
from sklearn.pipeline import Pipeline
import joblib

# Load the data
df = pd.read_csv('../data/prepared_data.csv') 
#df = df.drop(columns=['Loan_Status'],inplace=False)
#model = joblib.load("final_model.sav")  
#model.predict()

df.head()


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,146.412162,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.000000,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.000000,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.000000,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.000000,360.0,1.0,Urban,Y


In [16]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Define options for categorical variables
options_dependents = [{'label': str(i), 'value': i} for i in range(0, 4)]
options_education = [{'label': 'Graduate', 'value': 'Graduate'}, {'label': 'Not Graduate', 'value': 'Not Graduate'}]
options_self_employed = [{'label': 'Yes', 'value': 'Yes'}, {'label': 'No', 'value': 'No'}]
options_credit_history = [{'label': 'Yes', 'value': 1}, {'label': 'No', 'value': 0}]
options_property_area = [{'label': 'Urban', 'value': 'Urban'}, {'label': 'Semiurban', 'value': 'Semiurban'}, {'label': 'Rural', 'value': 'Rural'}]

# Define color scheme
colors = {
    'background': '#f9f9f9',  # Light gray background
    'text': '#333333',        # Dark gray text color
    'accent': '#4CAF50',      # Green accent color
    'button': '#FF5733',      # Orange button color
}

app.layout = html.Div(style={'backgroundColor': colors['background'], 'padding': '20px'}, children=[
    html.H1("Loan Approval Prediction", style={'color': colors['text'], 'textAlign': 'center'}),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Gender', style={'color': colors['text']}),
        dcc.Dropdown(
            id='gender-dropdown',
            options=[
                {'label': 'Male', 'value': 'Male'},
                {'label': 'Female', 'value': 'Female'}
            ],
            value='Male'
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Married', style={'color': colors['text']}),
        dcc.Dropdown(
            id='married-dropdown',
            options=[
                {'label': 'Yes', 'value': 'Yes'},
                {'label': 'No', 'value': 'No'}
            ],
            value='Yes'
        ),
    ]),
    # Add dropdowns for other features...
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Dependents', style={'color': colors['text']}),
        dcc.Dropdown(
            id='dependents-dropdown',
            options=options_dependents,
            value='0'
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Education', style={'color': colors['text']}),
        dcc.Dropdown(
            id='education-dropdown',
            options=options_education,
            value='Graduate'
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Self Employed', style={'color': colors['text']}),
        dcc.Dropdown(
            id='self-employed-dropdown',
            options=options_self_employed,
            value='No'
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Applicant Income: ', style={'color': colors['text']}),
        dcc.Input(
            id='applicant-income-input',
            type='number',
            value=0
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Coapplicant Income: ', style={'color': colors['text']}),
        dcc.Input(
            id='coapplicant-income-input',
            type='number',
            value=0
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Loan Amount: ', style={'color': colors['text']}),
        dcc.Input(
            id='loan-amount-input',
            type='number',
            value=0
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Loan Amount Term: ', style={'color': colors['text']}),
        dcc.Input(
            id='loan-amount-term-input',
            type='number',
            value=0
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Credit History', style={'color': colors['text']}),
        dcc.Dropdown(
            id='credit-history-dropdown',
            options=options_credit_history,
            value=1
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Property Area', style={'color': colors['text']}),
        dcc.Dropdown(
            id='property-area-dropdown',
            options=options_property_area,
            value='Urban'
        ),
    ]),
    
    html.Button('Predict', id='predict-button', n_clicks=0, style={'backgroundColor': colors['button'], 'color': '#FFFFFF', 'marginTop': '20px'}),
    
    html.Div(id='output-prediction', style={'marginTop': '20px', 'fontWeight': 'bold', 'fontSize': '18px', 'color': colors['accent'], 'textAlign': 'center'})
])


# Load Model
model = joblib.load('../artifacts/final_model.sav')

# Define callback function to handle user input and generate prediction
@app.callback(
    Output('output-prediction', 'children'),
    [Input('predict-button', 'n_clicks')],
    [dash.dependencies.State('gender-dropdown', 'value'),
     dash.dependencies.State('married-dropdown', 'value'),
     dash.dependencies.State('dependents-dropdown', 'value'),
     dash.dependencies.State('education-dropdown', 'value'),
     dash.dependencies.State('self-employed-dropdown', 'value'),
     dash.dependencies.State('applicant-income-input', 'value'),
     dash.dependencies.State('coapplicant-income-input', 'value'),
     dash.dependencies.State('loan-amount-input', 'value'),     
     dash.dependencies.State('loan-amount-term-input', 'value'),         
     dash.dependencies.State('credit-history-dropdown', 'value'),
     dash.dependencies.State('property-area-dropdown', 'value')]
)
def predict_loan_approval(n_clicks, gender, married, dependents, education, self_employed, applicantIncome, coapplicant_income, loan_amount, loan_amount_term, credit_history, property_area):
    print("Gender:", gender)
    print("Married:", married)
    print("Dependents:", dependents)
    print("Education:", education)
    print("Self Employed:", self_employed)
    print("Applicant Income:", applicantIncome)
    print("Coapplicant Income:", coapplicant_income)
    print("Loan Amount:", loan_amount)
    print("Loan Amount Term:", loan_amount_term)
    print("Credit History:", credit_history)
    print("Property Area:", property_area)
    
    input_data = pd.DataFrame({
        'Gender': [gender],
        'Married':[married],
        'Dependents': [dependents],
        'Education': [education],
        'Self_Employed': [self_employed],
        'ApplicantIncome': [applicantIncome],
        'CoapplicantIncome': [coapplicant_income],
        'LoanAmount': [loan_amount],
        'Loan_Amount_Term': [loan_amount_term],
        'Credit_History': [credit_history],
        'Property_Area': [property_area]
    })
    
    # Make predictions
    prediction = model.predict(input_data)
    prediction_result = 'Approved' if prediction[0] == "Y" else 'Rejected'
    return f"Prediction: {prediction_result}"


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


Gender: Male
Married: Yes
Dependents: 0
Education: Graduate
Self Employed: No
Applicant Income: 0
Coapplicant Income: 0
Loan Amount: 0
Loan Amount Term: 0
Credit History: 1
Property Area: Urban
Gender: Male
Married: Yes
Dependents: None
Education: Graduate
Self Employed: No
Applicant Income: 0
Coapplicant Income: 0
Loan Amount: 0
Loan Amount Term: 0
Credit History: 1
Property Area: Urban
Gender: Male
Married: Yes
Dependents: None
Education: Graduate
Self Employed: No
Applicant Income: 0
Coapplicant Income: 0
Loan Amount: 0
Loan Amount Term: 0
Credit History: 1
Property Area: Urban
Gender: Male
Married: Yes
Dependents: None
Education: Graduate
Self Employed: No
Applicant Income: 0
Coapplicant Income: 0
Loan Amount: 0
Loan Amount Term: 0
Credit History: 1
Property Area: Urban
Gender: Male
Married: Yes
Dependents: None
Education: Graduate
Self Employed: No
Applicant Income: 0
Coapplicant Income: 0
Loan Amount: 0
Loan Amount Term: 0
Credit History: 1
Property Area: Urban
Gender: Male
Marri